In [29]:
!nvidia-smi

Tue Sep 15 05:16:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    38W / 250W |    423MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import optimizers,regularizers
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from sklearn import metrics

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28, 1)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [32]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(MaxPool2D())
model.add(BatchNormalization()) 

model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=8,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(BatchNormalization()) 

model.add(Flatten())
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

adam = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 16)        4624      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 9, 9, 8)           1160      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 4, 4, 8)           0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 4, 4, 8)          

In [34]:
def scheduler(epoch):
  if epoch > 10 and epoch<=15:
    return 0.0005
  elif epoch > 15:
    return 0.0001
  else:
    return 0.001

import datetime
!rm -rf ./logs/ 
#tf.reset_default_graph()
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint('digit_classifier.h5', monitor='val_accuracy', verbose=1,\
                               save_best_only=True, save_weights_only=False, mode='max', period=1)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
LR_sched = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [37]:
history = model.fit(x_train,y_train,
                    batch_size=16,
                    epochs = 30,
                    use_multiprocessing=True, 
                    workers=4,
                    validation_data = (x_test,y_test),
                    callbacks = [tensorboard_callback,checkpoint,LR_sched],
                    # initial_epoch=20,
                    )

Epoch 21/30
3741/3750 [============================>.] - ETA: 0s - loss: 0.0386 - accuracy: 0.9870
Epoch 00021: val_accuracy improved from 0.99040 to 0.99070, saving model to digit_classifier.h5
3750/3750 [==============================] - 12s 3ms/step - loss: 0.0386 - accuracy: 0.9869 - val_loss: 0.0334 - val_accuracy: 0.9907
Epoch 22/30
3746/3750 [============================>.] - ETA: 0s - loss: 0.0399 - accuracy: 0.9861
Epoch 00022: val_accuracy did not improve from 0.99070
3750/3750 [==============================] - 13s 3ms/step - loss: 0.0399 - accuracy: 0.9861 - val_loss: 0.0331 - val_accuracy: 0.9899
Epoch 23/30
3743/3750 [============================>.] - ETA: 0s - loss: 0.0417 - accuracy: 0.9856
Epoch 00023: val_accuracy did not improve from 0.99070
3750/3750 [==============================] - 12s 3ms/step - loss: 0.0417 - accuracy: 0.9855 - val_loss: 0.0338 - val_accuracy: 0.9898
Epoch 24/30
3749/3750 [============================>.] - ETA: 0s - loss: 0.0389 - accuracy: 0.9

In [43]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)
y_pred.shape

(10000,)

In [45]:
acc = metrics.accuracy_score(y_test,y_pred,average='weighted')
acc

TypeError: ignored

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
!cp digit_classifier.h5 /content/drive/My\ Drive/